# SMS Spam Classification with PySpark and NLP

# Libraries

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# Loading Data

In [6]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [7]:
data = spark.read.csv("C:\Users\ivanm\Downloads\SMSSpamCollection", inferSchema=True, sep='\t')

In [8]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [9]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

In [10]:
data = data.withColumn('length',length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [12]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [13]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

In [15]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [16]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [17]:
cleaner = data_prep_pipe.fit(data)

In [18]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [19]:
clean_data = clean_data.select(['label','features'])

In [20]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [21]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [22]:
spam_predictor = nb.fit(training)

In [23]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [24]:
test_results = spam_predictor.transform(testing)

In [25]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-792.50720396145...|[1.0,1.5221421188...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-571.96780687169...|[1.0,4.9000492891...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-826.15123522298...|[1.0,1.3522081597...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.33266954339...|[1.0,3.3169040853...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-804.93840522338...|[1.0,8.8699480566...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-877.39768648922...|[1.0,2.9781219562...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1367.8900723800...|[1.0,1.8723908223...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-690.31809710157...|[1.0,7.9938272512...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-984.90322043095...|[0.99999999999852...|       0.0|
|  0.0|(13424,[0

In [26]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9288988867372427
